In [3]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import spicy
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense



key_day = "DDR"
data_dir = f'../data/data{key_day}'
dataset_dir = f'../data/dataset{key_day}'

##ESCOGER MENOR TAMAÑO DE CARPETA

folders = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]
num_hermandades = len(folders)
tamanos =[]
for folder in folders:
    folder_path = os.path.join(data_dir, folder)
    file_count = len(os.listdir(folder_path))
    tamanos.append(file_count)

menor = min(tamanos)
print(menor)

##LIMITAR A MENOR TAMAÑO DE CARPETA
carpeta_fuente = data_dir
carpeta_destino = dataset_dir

carpetas = os.listdir(carpeta_fuente)

for carpeta in carpetas:
    if not carpeta.startswith("."):
        imagenes = os.listdir(carpeta_fuente + '/' + carpeta)

        for i, nombreimg in enumerate(imagenes):
            if i < 42:
                if not nombreimg.startswith("."):
                    shutil.copy(carpeta_fuente + '/' + carpeta + '/' + nombreimg, carpeta_destino + '/' + carpeta + '/' + nombreimg)

##CREAR GENERADORES DE IMAGENES CON DATA AUGMENTATION
datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 30,
    zoom_range = [0.5, 1.5],
    validation_split=0.2 #20% para pruebas
)

##GENERADORES PARA ENTRENAMIENTO Y PRUEBAS
data_gen_entrenamiento = datagen.flow_from_directory(dataset_dir, target_size=(224,224),
                                                     batch_size=32, shuffle=True, subset='training')
data_gen_pruebas = datagen.flow_from_directory(dataset_dir, target_size=(224,224),
                                                     batch_size=32, shuffle=True, subset='validation')

42
Found 237 images belonging to 7 classes.
Found 56 images belonging to 7 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Input, Concatenate, GlobalAveragePooling2D

input_imagen = Input(shape=(224, 224, 3), name='input_imagen')

# Entrada para el día de la semana
input_dia_semana = Input(shape=(1,), name='input_dia_semana')

# Cargar el modelo InceptionV3
inception = InceptionV3(weights='imagenet', include_top=False, input_tensor=input_imagen)
inception.trainable = False

# Salida del modelo InceptionV3
inception_output = inception(input_imagen)

# Global Average Pooling en la salida del modelo InceptionV3
global_avg_pooling_inception = GlobalAveragePooling2D()(inception_output)

# Concatenar la salida de InceptionV3 con la entrada del día de la semana
concatenated_output = Concatenate()([global_avg_pooling_inception, input_dia_semana])

# Capa densa para procesar la información combinada
dense_combined = Dense(512, activation='relu')(concatenated_output)

# Capa de salida para la clasificación final
output_layer = Dense(num_hermandades, activation='softmax')(dense_combined)

# Crear el modelo final
modelo_final = tf.keras.Model(inputs=[input_imagen, input_dia_semana], outputs=output_layer)

# Compilar el modelo
modelo_final.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

EPOCAS = 20
data_dia = np.array([[0]])


'''
# Continuar entrenando el modelo con datos de MARTES
historial_inception_martes = modelo_inception.fit(
    {'input_imagen': data_gen_entrenamiento, 'input_dia_semana': key_day},
    epochs=EPOCAS,
    batch_size=32,
    validation_data=(
        {'input_imagen': data_gen_pruebas, 'input_dia_semana': key_day},
        data_gen_pruebas[1]
    )
)

# Guardar el modelo entrenado con datos de MARTES
save_model(modelo_inception, f'../models/{key_day}/{key_day}INCEPTION{EPOCAS}_MARTES.h5')
'''

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 5, 5, 2048), (None, 1)]

In [ ]:
# Entrenar el modelo final con datos de entrenamiento
historial_modelo_final = modelo_final.fit(
    {'input_imagen': data_gen_entrenamiento, 'input_dia_semana': data_dia},
    epochs=EPOCAS,
    batch_size=32,
    validation_data=(
        {'input_imagen': data_gen_pruebas, 'input_dia_semana': data_dia},
    )
)

In [ ]:
# Entrenar el modelo con datos de LUNES
historial_inception_lunes = modelo_inception.fit(
    {'input_imagen': data_gen_entrenamiento, 'input_dia_semana': data_dia},
    epochs=EPOCAS,
    batch_size=32,
    validation_data=(
        {'input_imagen': data_gen_pruebas, 'input_dia_semana': data_dia}
    )
)

# Guardar el modelo entrenado con datos de LUNES
save_model(modelo_inception, f'../models/FULLINCEPTION{EPOCAS}_LUNES.h5')
